In [5]:
import pandas as pd
import spotipy
import spotipy.util as util
import sys
sys.path.append('data/')
from spotify_client_details import *

In [7]:
spotify_raw = pd.read_json('data/StreamingHistory0.json')

In [8]:
songs = spotify_raw[['artistName','trackName']].drop_duplicates()
songs1 = songs.head(50)

In [9]:
from spotipy.oauth2 import SpotifyOAuth
scope = "user-library-read"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope, client_id=CLIENT_ID, 
                                               client_secret=CLIENT_SECRET, redirect_uri='http://localhost/'))


In [10]:
%time
artist = 'Led Zepellin'
track = "Stairway to Heaven"

a = sp.search(q=artist + ' ' + track)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11.4 µs


In [11]:
sp.audio_features(['36QJpDe2go2KgaRleHCDTp'])

[None]

In [12]:
print(a['tracks']['items'][0],'\n')
print(a['tracks']['items'][0]['album'].keys())
print(a['tracks']['items'][0]['album']['album_type'])

{'album': {'album_type': 'album', 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/36QJpDe2go2KgaRleHCDTp'}, 'href': 'https://api.spotify.com/v1/artists/36QJpDe2go2KgaRleHCDTp', 'id': '36QJpDe2go2KgaRleHCDTp', 'name': 'Led Zeppelin', 'type': 'artist', 'uri': 'spotify:artist:36QJpDe2go2KgaRleHCDTp'}], 'available_markets': ['AD', 'AE', 'AG', 'AL', 'AM', 'AO', 'AR', 'AT', 'AU', 'AZ', 'BA', 'BB', 'BD', 'BE', 'BF', 'BG', 'BH', 'BI', 'BJ', 'BN', 'BO', 'BR', 'BS', 'BT', 'BW', 'BY', 'BZ', 'CA', 'CH', 'CI', 'CL', 'CM', 'CO', 'CR', 'CV', 'CW', 'CY', 'CZ', 'DE', 'DJ', 'DK', 'DM', 'DO', 'DZ', 'EC', 'EE', 'EG', 'ES', 'FI', 'FJ', 'FM', 'FR', 'GA', 'GB', 'GD', 'GE', 'GH', 'GM', 'GN', 'GQ', 'GR', 'GT', 'GW', 'GY', 'HK', 'HN', 'HR', 'HT', 'HU', 'ID', 'IE', 'IL', 'IN', 'IS', 'IT', 'JM', 'JO', 'JP', 'KE', 'KG', 'KH', 'KI', 'KM', 'KN', 'KR', 'KW', 'KZ', 'LA', 'LB', 'LC', 'LI', 'LK', 'LR', 'LS', 'LT', 'LU', 'LV', 'MA', 'MC', 'MD', 'ME', 'MG', 'MH', 'MK', 'ML', 'MN', 'MO', 'MR', 'M

In [13]:
def albumName(artist,track):
    print(artist,track)
    song = sp.search(q='artist:{} track:{}'.format(artist, track))
    if song['tracks']['items']==[]:
        song = sp.search(q=artist + ' ' + track)
    song = song['tracks']['items']
    if song==[]:
        return None, None, None, None, None, None
    else:
        return song[0]['album']['name'], song[0]['duration_ms'], song[0]['popularity'], song[0]['track_number'], song[0]['album']['total_tracks'], song[0]['album']['release_date']
    

In [14]:
%%time
songs[['albumName','duration_ms','popularity','track_no','tracks_in_album','album_release_date']] = \
songs.apply(lambda row: albumName(row.artistName, row.trackName), axis=1,  result_type="expand")

King Gizzard & The Lizard Wizard Searching...
King Gizzard & The Lizard Wizard The River
King Gizzard & The Lizard Wizard Rattlesnake
King Gizzard & The Lizard Wizard Melting
King Gizzard & The Lizard Wizard Sleep Drifter
King Gizzard & The Lizard Wizard Nuclear Fusion
King Gizzard & The Lizard Wizard I'm In Your Mind
King Gizzard & The Lizard Wizard Cellophane
King Gizzard & The Lizard Wizard I'm Not In Your Mind
King Gizzard & The Lizard Wizard I'm In Your Mind Fuzz
King Gizzard & The Lizard Wizard Head On/Pill
King Gizzard & The Lizard Wizard I’m Not a Man Unless I Have A Woman
King Gizzard & The Lizard Wizard God Is Calling Me Back Home
King Gizzard & The Lizard Wizard Billabong Valley
King Gizzard & The Lizard Wizard Anoxia
King Gizzard & The Lizard Wizard Doom City
King Gizzard & The Lizard Wizard Sketches Of Brunswick East I
King Gizzard & The Lizard Wizard Countdown
King Gizzard & The Lizard Wizard D-Day
King Gizzard & The Lizard Wizard Tezeta
King Gizzard & The Lizard Wizard L

In [15]:
spotify_full = spotify_raw.merge(songs,on=['artistName','trackName'])

In [16]:
spotify_full.to_csv('data/song_features.csv')